In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [3]:
fake = pd.read_csv('Datasets\Fake.csv')
true = pd.read_csv('Datasets\True.csv')

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Irteja\AppData\Local\Temp\ipykernel_14152\3296890919.py:1: SyntaxWarning: invalid escape sequence '\F'
  fake = pd.read_csv('Datasets\Fake.csv')
C:\Users\Irteja\AppData\Local\Temp\ipykernel_14152\3296890919.py:2: SyntaxWarning: invalid escape sequence '\T'
  true = pd.read_csv('Datasets\True.csv')


In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class'] = 0
true['class'] = 1

In [7]:
df = pd.concat([fake, true]).sample(frac=1, random_state=42)

In [8]:
# Combine title + text into one column
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('') + " " + df['subject'].fillna('')

In [9]:
df.sample(10)

,title,text,subject,date,class,content
18299,LEFT-WING AUTHOR Blasts Democrats for “Scandal...,While many on the left continue to accuse Pres...,left-news,"Jul 18, 2017",0,LEFT-WING AUTHOR Blasts Democrats for “Scandal...
9866,Maxine Waters Hijacks Funeral To Bash Trump…Tu...,Maxine Waters got downright nasty at the funer...,politics,"Sep 20, 2017",0,Maxine Waters Hijacks Funeral To Bash Trump…Tu...
8543,Obama cuts short prison sentences for 214 conv...,WASHINGTON (Reuters) - President Barack Obama ...,politicsNews,"August 3, 2016",1,Obama cuts short prison sentences for 214 conv...
7041,Hillary Responds To Bernie Calling Her Unqual...,It seems there s a bit of infighting happening...,News,"April 7, 2016",0,Hillary Responds To Bernie Calling Her Unqual...
19534,U.S. will consider resuming halted military ai...,NEW YORK (Reuters) - The United States will co...,worldnews,"September 20, 2017",1,U.S. will consider resuming halted military ai...
6963,Hundreds Arrested At Democracy Spring Protest...,"Today, several Democracy Spring protesters w...",News,"April 11, 2016",0,Hundreds Arrested At Democracy Spring Protest...
19876,At least four killed in British motorway crash...,LONDON (Reuters) - At least four people died w...,worldnews,"September 16, 2017",1,At least four killed in British motorway crash...
19523,"YIKES! THE CLAWS COME OUT On The View, As Host...",Jill Stein agreed to do an interview with The ...,left-news,"Dec 1, 2016",0,"YIKES! THE CLAWS COME OUT On The View, As Host..."
8894,‘Family Values’ Republicans Introduce A Bill ...,You have to love the Republican party. They ca...,News,"January 7, 2016",0,‘Family Values’ Republicans Introduce A Bill ...
16223,Greek police arrest suspect behind high-profil...,ATHENS (Reuters) - Greek police said on Saturd...,worldnews,"October 28, 2017",1,Greek police arrest suspect behind high-profil...


In [10]:
df = df.drop(["title","subject","date","text"], axis = 1)

In [11]:
df.head()

,class,content
22216,0,Ben Stein Calls Out 9th Circuit Court: Committ...
4436,1,Trump drops Steve Bannon from National Securit...
1526,1,Puerto Rico expects U.S. to lift Jones Act shi...
1377,0,OOPS: Trump Just Accidentally Confirmed He Le...
8995,1,Donald Trump heads for Scotland to reopen a go...


In [12]:
df.reset_index(inplace=True)

In [15]:
df['cleaned'] = df['content'].apply(clean_text)

In [16]:
df.sample(10)

,index,class,content,cleaned
14367,17979,0,"KID ROCK GOES OFF On Colin Kaepernick, Obamaca...",kid rock goes off on colin kaepernick obamacar...
32791,16980,0,ILLEGAL ALIENS DEMAND NEW BILL OF RIGHTS: To I...,illegal aliens demand new bill of rights to in...
13641,3410,1,U.S. top court's Gorsuch says does not share '...,us top courts gorsuch says does not share cyni...
9089,5219,0,Crowd Erupts In Boos When Military Mom Asks P...,crowd erupts in boos when military mom asks pe...
2293,17406,1,U.S. officials try to ease concerns Trump may ...,us officials try to ease concerns trump may qu...
31676,23215,0,"SOPA False Flag? Alleged ‘Hack’ on Netflix, Tw...",sopa false flag alleged ‘hack’ on netflix twit...
28608,3800,0,"A Trump Fan Picks A Fight On A Plane, So The ...",a trump fan picks a fight on a plane so the ca...
29656,5203,0,Tea Party Candidate Wants To Be Elected So Ba...,tea party candidate wants to be elected so bad...
42214,6304,0,Marco Rubio SLAMS The GOP For Obstructing Oba...,marco rubio slams the gop for obstructing obam...
44244,6873,1,California Democrats propose series of infrast...,california democrats propose series of infrast...


In [14]:
# --- Text Cleaning ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [17]:
x =df["cleaned"]
y=df["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [18]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [19]:
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(xv_train, ytrain)

RandomForestClassifier(n_estimators=200, random_state=42)

In [20]:
prediction = model.predict(xv_test)
model.score(xv_test,ytest)

0.9929621380846325

In [21]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5863
           1       0.99      0.99      0.99      5362

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [22]:
joblib.dump(vectorizer,"vectorizer_rf.jb")
joblib.dump(model, "rf_model.jb")

['rf_model.jb']